# Word Embedding

In [1]:
import torch

## Word2Vec (2013)

[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)

by Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean

![](http://i.imgur.com/agTBWiT.png)

### Continuous Bag-Of-Words vs. Skip-gram

* CBOW: guessing the blank
* Skip-gram: guessing the neighbors

![](https://ascelibrary.org/cms/attachment/83d45b70-be2d-4dae-a37a-e3b51af0b7c4/figure3.jpg)

In [2]:
NANO_CORPUS = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

In [3]:
corpus = NANO_CORPUS.lower().replace(',', ' ').replace('.', ' ').split()
print(corpus)

['we', 'are', 'about', 'to', 'study', 'the', 'idea', 'of', 'a', 'computational', 'process', 'computational', 'processes', 'are', 'abstract', 'beings', 'that', 'inhabit', 'computers', 'as', 'they', 'evolve', 'processes', 'manipulate', 'other', 'abstract', 'things', 'called', 'data', 'the', 'evolution', 'of', 'a', 'process', 'is', 'directed', 'by', 'a', 'pattern', 'of', 'rules', 'called', 'a', 'program', 'people', 'create', 'programs', 'to', 'direct', 'processes', 'in', 'effect', 'we', 'conjure', 'the', 'spirits', 'of', 'the', 'computer', 'with', 'our', 'spells']


In [4]:
import pandas as pd

vocabulary = list(set(corpus))

/Users/dahlmoon/anaconda/envs/pytorch-nlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/dahlmoon/anaconda/envs/pytorch-nlp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Before we begin anything, we need to create a one-hot vector of the words. Pandas is great at this.

In [5]:
one_hot = pd.get_dummies(vocabulary)

In [6]:
EMBEDDING_SIZE = 128

class CBOW(torch.nn.Module):
    def __init__(self):
        super(CBOW, self).__init__()
        self.embeddings = torch.FloatTensor(len(vocabulary), EMBEDDING_SIZE).uniform_()
        self.linear1 = torch.nn.Linear(EMBEDDING_SIZE, 128)
        self.relu1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(128, len(vocabulary))
    
    def forward(self, x):
        x = torch.sum(self.embeddings * x.sum(dim=0).view(-1, 1), dim=0)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        
        return x.view(1, -1)
    
    def get_word_embedding(self, word):
        return self.embeddings[vocabulary.index(word)].view(1, -1)

cbow = CBOW()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cbow.parameters(), lr=0.001)

In [7]:
EPOCHS = 64
WINDOW_SIZE = 2
EMBEDDING_SIZE = 128

def get_context(i, corpus):
    context = []
    
    start = max(i - WINDOW_SIZE, 0)
    end = min(i + WINDOW_SIZE, len(corpus) - 1)
    
    for n in range(start, end):
        if n == i:
            continue
        context.append(corpus[n])
    
    return context

for epoch in range(EPOCHS):
    n_words = 0
    acc_loss = 0
    for i, word in enumerate(corpus):
        context = torch.FloatTensor(
            [one_hot[word] for word in get_context(i, corpus)])
        target = torch.LongTensor([vocabulary.index(word)])

        with torch.set_grad_enabled(True):
            output = cbow(context)
            loss = criterion(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            acc_loss += float(loss)
            n_words += 1

    print(f'Epoch {epoch}: loss {acc_loss/n_words:.4f}')

Epoch 0: loss 3.8499
Epoch 1: loss 3.7850
Epoch 2: loss 3.7396
Epoch 3: loss 3.7066
Epoch 4: loss 3.6820
Epoch 5: loss 3.6621
Epoch 6: loss 3.6450
Epoch 7: loss 3.6296
Epoch 8: loss 3.6153
Epoch 9: loss 3.6026
Epoch 10: loss 3.5902
Epoch 11: loss 3.5783
Epoch 12: loss 3.5671
Epoch 13: loss 3.5559
Epoch 14: loss 3.5446
Epoch 15: loss 3.5336
Epoch 16: loss 3.5229
Epoch 17: loss 3.5120
Epoch 18: loss 3.5011
Epoch 19: loss 3.4902
Epoch 20: loss 3.4792
Epoch 21: loss 3.4683
Epoch 22: loss 3.4570
Epoch 23: loss 3.4455
Epoch 24: loss 3.4344
Epoch 25: loss 3.4232
Epoch 26: loss 3.4123
Epoch 27: loss 3.4006
Epoch 28: loss 3.3889
Epoch 29: loss 3.3773
Epoch 30: loss 3.3656
Epoch 31: loss 3.3539
Epoch 32: loss 3.3420
Epoch 33: loss 3.3305
Epoch 34: loss 3.3181
Epoch 35: loss 3.3065
Epoch 36: loss 3.2944
Epoch 37: loss 3.2824
Epoch 38: loss 3.2703
Epoch 39: loss 3.2572
Epoch 40: loss 3.2447
Epoch 41: loss 3.2317
Epoch 42: loss 3.2191
Epoch 43: loss 3.2064
Epoch 44: loss 3.1930
Epoch 45: loss 3.179

Now, remember our corpus?

> We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create **programs** to direct processes. In effect,
we conjure the spirits of the computer with our spells.

Let's see if our model can guess the highlighted word.

In [8]:
quiz = ['people', 'create', 'to', 'direct']
output = cbow(torch.FloatTensor([one_hot[w] for w in quiz]))
_, i = output.max(dim=1)
print(vocabulary[i])

the


In [9]:
cbow.get_word_embedding('programs')

tensor([[0.3643, 0.9996, 0.9855, 0.7267, 0.1250, 0.0801, 0.6539, 0.8030, 0.7818,
         0.1762, 0.4257, 0.6482, 0.1161, 0.7514, 0.8206, 0.5402, 0.2067, 0.7947,
         0.2767, 0.5260, 0.9455, 0.9978, 0.4868, 0.7358, 0.5265, 0.7723, 0.2194,
         0.9570, 0.6649, 0.1071, 0.0101, 0.5208, 0.3882, 0.1479, 0.1548, 0.0120,
         0.2436, 0.2365, 0.8461, 0.6787, 0.6469, 0.6248, 0.2029, 0.8726, 0.9375,
         0.6713, 0.7686, 0.1980, 0.6803, 0.4200, 0.0986, 0.4354, 0.4928, 0.3195,
         0.9612, 0.0345, 0.2280, 0.6683, 0.5000, 0.3991, 0.1407, 0.9007, 0.5229,
         0.8227, 0.4821, 0.5555, 0.3531, 0.3194, 0.5222, 0.3366, 0.3674, 0.0203,
         0.4767, 0.9929, 0.2805, 0.3423, 0.2213, 0.0907, 0.4727, 0.2982, 0.3968,
         0.0270, 0.1012, 0.4033, 0.0502, 0.0996, 0.9084, 0.3239, 0.1664, 0.1366,
         0.9388, 0.1347, 0.5621, 0.2409, 0.5498, 0.0001, 0.0381, 0.4114, 0.4277,
         0.3794, 0.2763, 0.8652, 0.3884, 0.6464, 0.1489, 0.4899, 0.9416, 0.5688,
         0.9544, 0.9141, 0.7

In [10]:
class Skipgram(torch.nn.Module):
    def __init__(self):
        super(Skipgram, self).__init__()
        self.embeddings = torch.FloatTensor(len(vocabulary), EMBEDDING_SIZE).normal_()
        self.linear1 = torch.nn.Linear(EMBEDDING_SIZE, 128)
        self.relu1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(128, len(vocabulary))
    
    def forward(self, x):
        x = self.embeddings[x]
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        return x.view(1, -1)
    
    def get_word_embedding(self, word):
        return self.embeddings[vocabulary.index(word)].view(1, -1)

skipgram = Skipgram()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(skipgram.parameters(), lr=0.01)

In [11]:
EPOCHS = 64
WINDOW_SIZE = 2
EMBEDDING_SIZE = 128

def get_context(i, corpus):
    context = []
    
    start = max(i - WINDOW_SIZE, 0)
    end = min(i + WINDOW_SIZE, len(corpus) - 1)
    
    for n in range(start, end):
        if n == i:
            continue
        context.append(corpus[n])
    
    return context

for epoch in range(EPOCHS):
    n_words = 0
    acc_loss = 0
    for i, word in enumerate(corpus):
        center = vocabulary.index(word)

        for word in get_context(i, corpus):
            context = torch.LongTensor([vocabulary.index(word)])

            with torch.set_grad_enabled(True):
                output = skipgram(center)
                loss = criterion(output, context)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                acc_loss += float(loss)
                n_words += 1

    print(f'Epoch {epoch}: loss {acc_loss/n_words:.4f}')

Epoch 0: loss 3.7855
Epoch 1: loss 3.4326
Epoch 2: loss 3.1628
Epoch 3: loss 2.9233
Epoch 4: loss 2.7064
Epoch 5: loss 2.5197
Epoch 6: loss 2.3735
Epoch 7: loss 2.2685
Epoch 8: loss 2.1979
Epoch 9: loss 2.1520
Epoch 10: loss 2.1259
Epoch 11: loss 2.1066
Epoch 12: loss 2.0931
Epoch 13: loss 2.0831
Epoch 14: loss 2.0732
Epoch 15: loss 2.0650
Epoch 16: loss 2.0570
Epoch 17: loss 2.0499
Epoch 18: loss 2.0427
Epoch 19: loss 2.0365
Epoch 20: loss 2.0306
Epoch 21: loss 2.0242
Epoch 22: loss 2.0184
Epoch 23: loss 2.0118
Epoch 24: loss 2.0060
Epoch 25: loss 1.9998
Epoch 26: loss 1.9950
Epoch 27: loss 1.9893
Epoch 28: loss 1.9849
Epoch 29: loss 1.9814
Epoch 30: loss 1.9768
Epoch 31: loss 1.9733
Epoch 32: loss 1.9675
Epoch 33: loss 1.9645
Epoch 34: loss 1.9599
Epoch 35: loss 1.9562
Epoch 36: loss 1.9527
Epoch 37: loss 1.9486
Epoch 38: loss 1.9455
Epoch 39: loss 1.9421
Epoch 40: loss 1.9380
Epoch 41: loss 1.9358
Epoch 42: loss 1.9322
Epoch 43: loss 1.9294
Epoch 44: loss 1.9261
Epoch 45: loss 1.921

In [12]:
def get_similar(query, embeddings, top_k=10):
    embeddings = embeddings.cpu()
    query = embeddings[vocabulary.index(query)]
    similarity = (embeddings @ query) / (embeddings.norm() * query.norm())
    similarity = pd.Series(dict(zip(vocabulary, similarity.numpy())))
    similarity = similarity.sort_values(ascending=False)
    
    return similarity[:top_k]

get_similar('people', skipgram.embeddings)

people       0.152117
things       0.031600
to           0.021331
computers    0.021299
called       0.018467
process      0.016660
directed     0.016640
are          0.013240
evolve       0.010218
is           0.010014
dtype: float64

## GloVe: Global Vectors for Word Representation (2014)

by Jeffrey Pennington, Richard Socher, Christopher D. Manning

https://www.aclweb.org/anthology/D14-1162

On page 1534:

> We begin with a simple example that showcases
how certain aspects of meaning can be extracted
directly from co-occurrence probabilities. Consider
two words $i$ and $j$ that exhibit a particular aspect
of interest; for concreteness, suppose we are
interested in the concept of thermodynamic phase,
for which we might take $i = ice$ and $j = steam$.
The relationship of these words can be examined
by studying the ratio of their co-occurrence probabilities
with various probe words, $k$. For words
$k$ related to $ice$ but not $steam$, say $k = solid$, we
expect the ratio $Pik / Pjk$ will be large. Similarly,
for words $k$ related to $steam$ but not $ice$, say $k =
gas$, the ratio should be small. For words $k$ like
$water$ or $fashion$, that are either related to both $ice$
and $steam$, or to neither, the ratio should be close
to one. Table 1 shows these probabilities and their
ratios for a large corpus, and the numbers confirm
these expectations. Compared to the raw probabilities,
the ratio is better able to distinguish relevant
words ($solid$ and $gas$) from irrelevant words ($water$
and $fashion$) and it is also better able to discriminate
between the two relevant words.

$$
\frac{P_{solid | ice}}{P_{solid | steam}} >
\frac{P_{fashion | ice}}{P_{fashion | steam}} >
\frac{P_{gas | ice}}{P_{gas | steam}}
$$

> The above argument suggests that the appropriate
starting point for word vector learning should
be with ratios of co-occurrence probabilities rather
than the probabilities themselves. Noting that the
ratio $P_{ik} /P_{jk}$ depends on three words $i$, $j$, and $k$,
the most general model takes the form,

$$
F(w_i, w_j, \tilde{w}_k) = \frac{P_{ik}}{P_{jk}}
$$

> The number of possibilities for $F$ is vast,
but by enforcing a few desiderata we can select a
unique choice. First, we would like $F$ to encode
the information present the ratio $Pik / Pjk$ in the
word vector space. Since vector spaces are inherently
linear structures, the most natural way to do
this is with vector differences.

$$
F(w_i - w_j, \tilde{w}_k) = \frac{P_{ik}}{P_{jk}}
$$

> Next, we note that the arguments of $F$ in Eqn. (2)
are vectors while the right-hand side is a scalar.
While $F$ could be taken to be a complicated function
parameterized by, e.g., a neural network, doing
so would obfuscate the linear structure we are
trying to capture. To avoid this issue, we can first
take the dot product of the arguments,

$$
F((w_i - w_j)^T \tilde{w}_k) = \frac{P_{ik}}{P_{jk}}
$$

> Next, note that for
word-word co-occurrence matrices, the distinction
between a word and a context word is arbitrary and
that we are free to exchange the two roles. To do so
consistently, we must not only exchange $w \leftrightarrow \tilde{w}$
but also $X \leftrightarrow X^T$. Our final model should be invariant
under this relabeling, but Eqn. (3) is not.
However, the symmetry can be restored in two
steps. First, we require that $F$ be a homomorphism
between the groups $(\mathbb{R}, +)$ and $(\mathbb{R}_{>0}, \times)$, i.e.,

$$
F(X-Y)=\frac { F(X) }{ F(Y) }
$$

$$
F(w_i^T \tilde{w}_k - w_j^T \tilde{w}_k) = \frac{P_{ik}}{P_{jk}}
$$

$$
F(w_i^T \tilde{w}_k - w_j^T \tilde{w}_k) = \frac{F(w_i^T \tilde{w}_k)}{F(w_j^T \tilde{w}_k)}
$$

$$
\exp(w_i^T \tilde{w}_k - w_j^T \tilde{w}_k) = \frac{\exp(w_i^T \tilde{w}_k)}{\exp(w_j^T \tilde{w}_k)}
$$

$$F = \exp$$

Page 1533:
> Let the matrix
of word-word co-occurrence counts be denoted by
$X$, whose entries $X_{ij}$ tabulate the number of times
word $j$ occurs in the context of word $i$. Let $X_i = \sum_k X_{ik}$
be the number of times any word appears
in the context of word $i$. Finally, let
$P_{ij} = P(j|i) = X_{ij}/X_i$be the probability that word $j$ appear in the
context of word $i$.

$$
F(w_i^T \tilde{w}_k) = P_{ik} = \frac{X_{ik}}{X_i}
$$

$$
w_i^T \tilde { w }_k =\log { { P }_{ ik } } =\log ({ X_{ ik })-\log ({ X_{ i } })  }
$$

Page 1535:
> Next, we note that Eqn. (6) would exhibit the exchange
symmetry if not for the $log(X_i)$ on the
right-hand side. 

$$
\log(X_{ik})-\log(X_i) \neq \log(X_{ki})-\log(X_k)
$$

> However, this term is independent
of $k$ so it can be absorbed into a bias $b_i$ for
$w+i$. Finally, adding an additional bias $\tilde{b}_k$ for $\tilde{w}_k$
restores the symmetry,

$$
{ w }_{ i }^{ T }\tilde { { w }_{ k } } +{ b }_{ i }+\tilde { { b }_{ k } } =\log ({ X_{ ik }})
$$

## Building the vocabulary and counting co-occurrence (again)

Today's dataset, an English monolingual corpus, can be found [here](https://drive.google.com/open?id=1__lK0x_k8gtyV27QZqQUGSC4jlaQAZSC).

In [14]:
from collections import defaultdict

FILE = 'ted.en.txt'
WINDOW_SIZE = 10

vocabulary = defaultdict(int)
co_occurrence = defaultdict(int)

with open(FILE) as f:
    sentences = f.readlines()

for sentence in sentences:
    words = sentence.split(' ')
    for i in range(len(words)):
        vocabulary[words[i]] += 1

        for j in range(i + 1, i + WINDOW_SIZE + 1):
            if j >= len(words):
                break
            keys = tuple(sorted([words[i], words[j]]))
            co_occurrence[keys] += 1

Let's see how much words we have gathered.

In [15]:
len(vocabulary)

77599

Show some love!

In [16]:
'love' in vocabulary

True

How much?

In [17]:
vocabulary['love']

2444

Let's convert the dictionary into a Pandas Series for convinience.

In [18]:
import pandas as pd

MIN_OCCURRENCE = 10

vocabulary = pd.Series(vocabulary, dtype='uint16')

And with the help of Pandas, let's set a minimum frequency threshold to trim the vocabulary.

In [19]:
vocabulary = vocabulary[vocabulary >= MIN_OCCURRENCE]
len(vocabulary)

16754

In [20]:
'love' in vocabulary

True

In [21]:
import numpy as np

X_ij = np.zeros((len(vocabulary), len(vocabulary)), dtype='uint16')

for (word_i, word_j), value in co_occurrence.items():
    try:
        i = vocabulary.index.get_loc(word_i)
        j = vocabulary.index.get_loc(word_j)
    except KeyError:
        continue

    X_ij[i][j] = value
    X_ij[j][i] = value

In [23]:
X_ij.shape

(16754, 16754)

$$
{ w }_{ i }^{ T }\tilde { { w }_{ k } } +{ b }_{ i }+\tilde { { b }_{ k } } =\log ({ X_{ ik }})
$$

In [22]:
from itertools import chain
import torch

DIM = 128
ITERATIONS = 32
X_MAX = 100
ALPHA = 3/4
GPU_ID = 2

n_words = X_ij.shape[0]

X = torch.from_numpy(X_ij.astype('float32'))
w_main = torch.FloatTensor(n_words, DIM).uniform_(-0.5, 0.5)
w_context = torch.FloatTensor(n_words, DIM).uniform_(-0.5, 0.5)
b_main = torch.FloatTensor(n_words).uniform_(-0.5, 0.5)
b_context = torch.FloatTensor(n_words).uniform_(-0.5, 0.5)

if torch.cuda.is_available():
    X = X.cuda(device=GPU_ID)
    w_main = w_main.cuda(device=GPU_ID)
    w_context = w_context.cuda(device=GPU_ID)
    b_main = b_main.cuda(device=GPU_ID)
    b_context = b_context.cuda(device=GPU_ID)

X.requires_grad_(False)
w_main.requires_grad_(True)
w_context.requires_grad_(True)
b_main.requires_grad_(True)
b_context.requires_grad_(True)

criterion = torch.nn.MSELoss(reduction='none')
optimizer = torch.optim.Adam([w_main, w_context, b_main, b_context],
                             lr=1e-3, weight_decay=1e-15)

with torch.set_grad_enabled(True):
    for iteration in range(ITERATIONS):
        acc_loss = 0
        for j in torch.randperm(n_words):
            output = w_main @ w_context[j]
            output += b_main
            output += b_context[j]
            
            loss = criterion(output, X[:, j].log() + 1e-15)
            
            loss_weight = (X[:, j] / X_MAX) ** ALPHA
            loss_weight[X[:, j] > X_MAX] = 1

            optimizer.zero_grad()
            loss.backward(loss_weight)
            optimizer.step()
            
            acc_loss += float(loss.mean())
        
        print(f'iteration {iteration}, loss {acc_loss/n_words:.4f}')

KeyboardInterrupt: 

In [ ]:
def v(word):
    i = vocabulary.index.get_loc(word)
    return w_main[i].cpu()

def analogy(target, top_k=20):
    target /= target.norm()
    
    with torch.no_grad():
        similarity = (w_main.cpu() @ target) / (w_main.cpu().norm() * target.norm())
        similarity = pd.Series(dict(zip(vocabulary.keys(), similarity.numpy())))
        similarity = similarity[vocabulary < 500]
        similarity = similarity.sort_values(ascending=False)
    
    return similarity.sort_values(ascending=False)[:top_k]

analogy(v('husband') - v('man') + v('woman'))

In [ ]:
analogy(v('heaven') - v('good') + v('bad'))

## Visualization using PCA and t-SNE

![](https://scontent-icn1-1.xx.fbcdn.net/v/t1.0-9/41425661_1809264752526756_3946431284045152256_n.jpg?_nc_cat=107&oh=e0b118959eaf0d6c7c97ce71b8c1136d&oe=5C20EDF1)

* PCA
  * good for dimensionality reduction
  * not always good for visualization
  * weak against non-linear data
* t-SNE
  * good for visualization
  * not so good for dimensionality reduction
  * strong against non-linear data

[FastText](https://fasttext.cc/): Library for efficient text classification and representation learning

In [ ]:
import torchtext
fasttext = torchtext.vocab.FastText(language='simple')

In [ ]:
fasttext['love']

In [ ]:
fasttext.vectors.size()

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

def pca(vocabulary, embeddings, n_points):
    np.random.seed(0)

    frequent = vocabulary[vocabulary < 2000].sort_values(ascending=False).index[:n_points]
    indices = [vocabulary.index.get_loc(word) for word in frequent]
    
    pca = PCA(n_components=2, random_state=0)
    with torch.no_grad():
        results = pca.fit_transform(embeddings[indices])
    
    plt.figure(figsize=(15, 15))
    for i in range(n_points):
        query = vocabulary.index[indices[i]]
        x, y = results[i]
        plt.scatter(x, y, label=query)
        
        # Prevent label overlapping by applying random offsets.
        offset_x = np.random.randint(-35, 12) / 2000
        offset_y = np.random.randint(-30, 15) / 2000
        
        plt.annotate(query, (x + offset_x, y + offset_y))
        
    plt.show()
    
pca(vocabulary, w_main, 100)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
    
def tsne(vocabulary, embeddings, n_points):
    np.random.seed(0)

    frequent = vocabulary[vocabulary < 2000].sort_values(ascending=False).index[:n_points]
    indices = [vocabulary.index.get_loc(word) for word in frequent]

    tsne = TSNE(n_components=2, random_state=0)
    with torch.no_grad():
        results = tsne.fit_transform(embeddings[indices])
    
    plt.figure(figsize=(15, 15))
    for i in range(n_points):
        query = vocabulary.index[indices[i]]
        x, y = results[i]
        plt.scatter(x, y, label=query)
        
        # Prevent label overlapping by applying random offsets.
        offset_x = np.random.randint(-35, 12) / 100
        offset_y = np.random.randint(-30, 15) / 100
        
        plt.annotate(query, (x + offset_x, y + offset_y))
        
    plt.show()

tsne(vocabulary, w_main, 200)